In [1]:
import sys
import numpy as np
import pandas as pd
import dask.dataframe as dd
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import collections
import optuna
from optuna.samplers import TPESampler

import feather
import os
import gc
import glob
import time

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve
from sklearn.metrics import make_scorer,f1_score, recall_score, accuracy_score, average_precision_score, auc
from sklearn.metrics import f1_score, mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score, precision_score

import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import pickle

import datatable as dt
from datatable import (dt, f, g, by, ifelse, update, sort, join,
                      count, min, max, mean, sum, rowsum)
from sklearn.preprocessing import LabelEncoder

In [2]:
def get_previous_timestamp(df):
    df = df[['key', 'timestamp']]
    keys, values = df.sort_values(['key']).values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    timestamp_list_dict = dict(zip(ukeys, arrays))
    x = np.ndarray(shape=(len(keys),), dtype=object)
    for idx, k in enumerate(tqdm(keys)):
        list_t = timestamp_list_dict.get(k).astype('int')
        x[idx] = list_t
    print("List of timestamp processed")
    data = get_lu_previous_timestamp(values, x, keys)
    return data

In [3]:
def get_lu_previous_timestamp(i, j, origin_keys):
    #i = np.array(data['timestamp'].tolist())
    #j = np.array(data['previous_timestamp'].tolist())
    d1 = i - j
    d0 = collections.OrderedDict(enumerate(i.flatten(), 1))
    d1 = collections.OrderedDict(enumerate(d1.flatten(), 1))
    d2 = collections.OrderedDict(enumerate(j.flatten(), 1))
    #print("d2 : {}".format(d2))
    #print("d0 : {}".format(d0))
    #print("d1 : {}".format(d1))
    values = []
    #min_values = []
    #flags = []
    #indexes = []
    #look = []
    #look_V2 = []
    #li = []
    #keys = []
    original_values = []
    for k,v in tqdm(d1.items()):
        values_d1 = v.tolist()
        #Calculate min value positive
        min_value = np.where(d1.get(k) > 0, d1.get(k), np.nan)
        min_value = np.nanmin(min_value)
        if np.isnan(min_value):
            flag = 1
            min_value = 0
        else:
            flag = 0
        min_value = int(min_value)
        #print("values_d1 : {}".format(values_d1))
        #print("type values_d1 : {}".format(type(values_d1)))
        #print("min_value : {}".format(min_value))
        #print("type min_value : {}".format(type(min_value)))
        index = values_d1.index(min_value)
        original_value = d0.get(k)
        lookup = d2.get(k)
        lookup_v2 = lookup[index]
        #Define nan when no previous timestamp ( new value = old value)
        final = np.where(flag == 1 , -1 , lookup[index])
        
        #Define final list of previous values
        values.append(final)
        #look.append(lookup)
        #li.append(values_d1)
        #min_values.append(min_value)
        #flags.append(flag)
        #indexes.append(index)
        #keys.append(k)
        #look_V2.append(lookup_v2)
        #original_values.append(original_value)

    #df2 = pd.DataFrame({'lu_previous_timestamp':values,'values_d1':li, 'min_values':min_values, 'flags':flags, 'indexes':indexes, 'lookup': look,'lookup_V2': look_V2, 'keys':keys, 'original_values':original_values })
    df2 = pd.DataFrame({'lu_previous_timestamp':values, 'key':origin_keys, 'timestamp':i})
    return (df2)

In [4]:
def merge__by_chunks_pandas(data, data2, key, chunksize):
    start = time.time()
    frames = {}
    chunk_size =  chunksize
    size_final_file = data.shape[0]
    cols = data.shape[1]
    nb_iterations = math.ceil((data.shape[0]/chunk_size))
    subset_start = 0
    print('Start iteractions over {} chunks to convert {:,} rows of data with {} columns'.format(nb_iterations, size_final_file, cols))
    print('Each chunk will process {:,} / {:,}'.format(chunksize, data.shape[0]))
    for i in tqdm(range(0,nb_iterations), position=0, leave=True):
        subset_end = subset_start + chunk_size
        print('subset_start : {}'.format(subset_start))
        print('subset_end : {}'.format(subset_end))
        print('Total Rows : {}'.format(data.shape[0]))
        if data.shape[0]<subset_end:
            subset_end = data.shape[0]
            print('subset end updated to remaining rows')
        if subset_end <= (data.shape[0]) :
            data.set_index(key, inplace=True, drop=False)
            #data2.set_index(key, inplace=True, drop=False)
            subset = data[subset_start:subset_end]
            print('chunk #{} size : {:,} rows'.format(str(i + 1), subset.shape[0]))
            subset = subset.merge(data2, how = 'left', left_index=True, right_index=True)
            subset['chunk'] = i
            frames[i] = subset
            print ('data load for chunk {}'.format(i + 1))
            subset_start =  subset_end
            step = time.time()
            print( "Time to process data load for chunk {} :  \n {:.2f} min".format( i + 1, (step - start) / 60))
    print( "Starting Aggregate pandas dataframes")
    if len(frames) > 1:
        # Concat all new dataframes
        final = pd.concat(frames.values(), ignore_index=True)
    else:
        final = frames.values()
    print ("Finished")
    return final

In [5]:
def process__by_list_chunks_pandas(data, iteration_list, iteration_column_name, function, chunksize):
    start = time.time()
    frames = {}
    chunk_size =  chunksize
    size_final_file = data.shape[0]
    cols = data.shape[1]
    nb_iterations = math.ceil((len(iteration_list)/chunk_size))
    subset_start = 0
    print('Start iteractions over {} chunks to convert {:,} rows of data with {} columns'.format(nb_iterations, size_final_file, cols))
    print('Each chunk will process {:,} / {:,} {}'.format(chunksize, len(iteration_list), iteration_column_name))
    for i in range(0,nb_iterations):
        subset_end = subset_start + chunk_size
        print('subset_start : {}'.format(subset_start))
        print('subset_end : {}'.format(subset_end))
        print('users_list_len : {}'.format(len(iteration_list)))
        if len(iteration_list)<subset_end:
            subset_end = len(iteration_list)
            print('subset end updated to remaining rows')
        if subset_end <= (len(iteration_list)) :
            user_list_chunk = iteration_list[subset_start:subset_end]
            subset = data[data[iteration_column_name].isin(user_list_chunk)]
            print('chunk #{} size : {:,} rows'.format(str(i + 1), subset.shape[0]))
            subset = function(subset)
            #subset['chunk'] = i
            frames[i] = subset
            print ('data load for chunk {}'.format(i + 1))
            subset_start =  subset_end
            step = time.time()
            print( "Time to process data load for chunk {} :  \n {:.2f} min".format( i + 1, (step - start) / 60))
    print( "Starting Aggregate pandas dataframes")
    if len(frames) > 1:
        # Concat all new dataframes
        final = pd.concat(frames.values(), ignore_index=True)
    else:
        final = frames.values()
    print ("Finished")
    return final

In [6]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [7]:
N_ROWS = 99271101
N_ROWS_train = 50004223
N_ROWS_test = 2500267
N_ROWS_user_stats = 393656
N_ROWS_content_stats = 13523
N_ROWS_previous_stats_questions = 86867031
def get_index_np(type_data):
    if type_data == 'train':
        final = np.arange(N_ROWS_train)
    if type_data == 'test':
        final = np.arange(N_ROWS_test)
    if type_data == 'user_stats':
        final = np.arange(N_ROWS_user_stats)
    if type_data == 'content_stats':
        final = np.arange(N_ROWS_content_stats)
    if type_data == 'previous_stats_questions':
        final = np.arange(N_ROWS_previous_stats_questions)
    if type_data == 'full_dataset':
        final = np.arange(N_ROWS)
    return final

In [8]:
def load_data_np(file_path, columns_selected, type_data):
    data = np.load(file_path , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np(type_data))
    for col in tqdm(columns_selected):
        df[col] = data[col]
    return df

## Loading Dataset 

In [ ]:
%%time

# reading the dataset from raw csv file
data = dt.fread("/Users/nlisch/downloads/Kaggle_riid/train.csv")
print(data.shape)

In [ ]:
data = data.to_pandas()
#data['prior_question_had_explanation'] = data['prior_question_had_explanation'].astype('bool')

In [ ]:
data.head(10)

## Data cleaning 

In [ ]:
%%time
#Drop dupplicated rows based on specific columns
data.drop_duplicates(subset=['user_id', 'content_id', 'timestamp'], inplace = True)

In [ ]:
# Add questions to train
questions = pd.read_csv('/Users/nlisch/downloads/Kaggle_riid/questions.csv')

In [ ]:
questions.groupby(['part']).agg({'question_id': ['count','skew', 'mean', 'std', 'median'] }).reset_index()

In [ ]:
data = data.merge(questions, how = 'left', left_on = 'content_id', right_on = 'question_id')
del questions
gc.collect()

In [ ]:
%%time
# Remove lectures rows
data = data[data['answered_correctly']!= -1]

data['prior_question_elapsed_time'].fillna(23916, inplace=True)

In [ ]:
%%time

# Create question_answered column
data.loc[data.answered_correctly != -1, 'question_answered' ] = 1

# Create false answer column
data.loc[data.answered_correctly == 0, 'false_answer' ] = 1
data['false_answer'].fillna(0, inplace=True)

In [ ]:
%%time
#Create Key columns
data['key'] = data['user_id'].astype(str) + '-' + data['content_id'].astype(str)

In [ ]:
%%time
# Sort value for previous question timestamp
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
map_prior = {True:1, False:0}
data["prior_answer"] = data["prior_question_had_explanation"].map(map_prior)

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
row_id = np.array(data['row_id'].tolist())
timestamp = np.array(data['timestamp'].tolist())
user_id = np.array(data['user_id'].tolist())
content_id = np.array(data['content_id'].tolist())
content_type_id = np.array(data['content_type_id'].tolist())
task_container_id = np.array(data['task_container_id'].tolist())
user_answer = np.array(data['user_answer'].tolist())
answered_correctly = np.array(data['answered_correctly'].tolist())
prior_question_elapsed_time = np.array(data['prior_question_elapsed_time'].tolist())
prior_question_had_explanation = np.array(data['prior_question_had_explanation'].tolist())
question_id = np.array(data['question_id'].tolist())
bundle_id = np.array(data['bundle_id'].tolist())
correct_answer = np.array(data['correct_answer'].tolist())
part = np.array(data['part'].tolist())
tags = np.array(data['tags'].tolist())
question_answered = np.array(data['question_answered'].tolist())
false_answer = np.array(data['false_answer'].tolist())
key = np.array(data['key'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz',
                    row_id = row_id,
                    timestamp = timestamp,
                    user_id = user_id,
                    content_id = content_id,
                    content_type_id = content_type_id,
                    task_container_id = task_container_id,
                    user_answer = user_answer,
                    answered_correctly = answered_correctly,
                    prior_question_elapsed_time = prior_question_elapsed_time,
                    prior_question_had_explanation = prior_question_had_explanation,
                    question_id = question_id,
                    bundle_id = bundle_id,
                    correct_answer = correct_answer,
                    part = part,
                    tags = tags,
                    question_answered = question_answered,
                    false_answer = false_answer, 
                    key=key
                   )

del row_id
del timestamp
del user_id
del content_id
del content_type_id
del task_container_id
del user_answer
del answered_correctly
del prior_question_elapsed_time
del prior_question_had_explanation
del question_id
del bundle_id
del correct_answer
del part
del tags
del question_answered
del false_answer
del key

gc.collect()

In [ ]:
data.head(10)

# Timestamp list by question for each user

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['key', 'timestamp', 'user_id']
data = load_data_np(file_path, columns)

In [ ]:
#data['key'] = data['user_id'].astype(str) + '-' + data['content_id'].astype(str)
#data = data[['key', 'timestamp', 'user_id']]
#data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
users_list = data['user_id'].unique().tolist()

In [ ]:
%%time
list_timestamp_h_question = process__by_list_chunks_pandas(data = data,
                                  iteration_list = users_list,
                                  iteration_column_name = 'user_id',
                                  function = get_previous_timestamp,
                                  chunksize = 10000)

In [ ]:
%%time
list_timestamp_h_question = list_timestamp_h_question.sort_values(by=['key', 'timestamp'])

In [ ]:
%%time
#Convert pandas columns to array
key = np.array(list_timestamp_h_question['key'].tolist())
lu_previous_timestamp = np.array(list_timestamp_h_question['lu_previous_timestamp'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/list_timestamp_h_question.npz',
                    key=key,
                    lu_previous_timestamp=lu_previous_timestamp)
del key
del lu_previous_timestamp
gc.collect

In [ ]:
%%time
data['lu_previous_timestamp'] = lu_previous_timestamp

In [ ]:
%%time
data['key2'] = key

In [ ]:
data[data['key'] != data['key2']]

In [ ]:
data.sample(10)

In [ ]:
data[(data['user_id'] == 1434394909)]\
          .sort_values(by=['key','timestamp'])

## Previous stats

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['user_id', 'content_id', 'timestamp', 'answered_correctly', 'false_answer']
df_h_question = load_data_np(file_path, columns)

In [ ]:
df_h_question.shape

In [ ]:
%%time
df_h_question['previous_false_answer'] = df_h_question.sort_values(by=['user_id', 'content_id', 'timestamp'], ascending=True)\
                       .groupby(['user_id', 'content_id'])['false_answer']\
                       .cumsum()

In [ ]:
%%time
df_h_question['previous_correct_answer'] = df_h_question.sort_values(by=['user_id', 'content_id', 'timestamp'], ascending=True)\
                       .groupby(['user_id', 'content_id'])['answered_correctly']\
                       .cumsum()

In [ ]:
%%time
df_h_question['nb_previous_questions_answered'] = df_h_question.sort_values(['user_id', 'content_id', 'timestamp'], ascending=True)\
             .groupby(['user_id', 'content_id'])\
             .cumcount()

In [ ]:
%%time
df_h_question['previous_answer'] = df_h_question.sort_values(by=['user_id', 'content_id', 'timestamp'], ascending=True)\
                       .groupby(['user_id', 'content_id'])['answered_correctly'].shift(1)

In [ ]:
%%time
df_h_question['previous_answer'].fillna(999999, inplace=True)

df_h_question['previous_correct_answer'] = df_h_question['previous_correct_answer'].astype(int)
df_h_question['previous_false_answer'] = df_h_question['previous_false_answer'].astype(int)
df_h_question['previous_answer'] = df_h_question['previous_answer'].astype(int)
df_h_question['false_answer'] = df_h_question['false_answer'].astype(int)

In [ ]:
%%time
df_h_question.rename(columns = {'timestamp':'lu_previous_timestamp'}, inplace = True)

In [ ]:
df_h_question, NAlist = reduce_mem_usage(df_h_question)

In [ ]:
%%time
df_h_question['key'] = df_h_question['user_id'].astype(str) + '-' + df_h_question['content_id'].astype(str)
df_h_question['key'] = df_h_question['key'].astype(str) + '-' + df_h_question['lu_previous_timestamp'].astype(str)
df_h_question = df_h_question[['key',
                    'previous_false_answer',
                    'previous_correct_answer',
                    'nb_previous_questions_answered',
                    'previous_answer']]

In [ ]:
df_h_question.head(10)

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
key = np.array(df_h_question['key'].tolist())
previous_false_answer = np.array(df_h_question['previous_false_answer'].tolist())
previous_correct_answer = np.array(df_h_question['previous_correct_answer'].tolist())
nb_previous_questions_answered = np.array(df_h_question['nb_previous_questions_answered'].tolist())
previous_answer = np.array(df_h_question['previous_answer'].tolist())
del df_h_question
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/previous_answer_questions_stats.npz',
                    key=key,
                    previous_false_answer=previous_false_answer,
                    previous_correct_answer = previous_correct_answer,
                    nb_previous_questions_answered = nb_previous_questions_answered,
                    previous_answer = previous_answer
                   )
del key
del previous_false_answer
del previous_correct_answer
del nb_previous_questions_answered
del previous_answer
gc.collect()

In [ ]:
#MERGE previous questions stats with data

## Load data needed

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['key']
data = load_data_np(file_path, columns)

In [ ]:
%%time
lu_previous_timestamp = np.load('/Users/nlisch/downloads/Kaggle_riid/process/list_timestamp_h_question.npz' , allow_pickle=True)
data['lu_previous_timestamp'] = lu_previous_timestamp['lu_previous_timestamp']
del lu_previous_timestamp
gc.collect()

In [ ]:
lu_previous_timestamp['key'][7544719]

In [ ]:
lu_previous_timestamp['lu_previous_timestamp'][7544719]

In [ ]:
data.iloc[7544719]

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/previous_answer_questions_stats.npz'
columns = [ 'key',
            'previous_false_answer',
            'previous_correct_answer',
            'nb_previous_questions_answered',
            'previous_answer']
previous_answer_questions_stats = load_data_np(file_path, columns)

In [ ]:
previous_answer_questions_stats.head(10)

In [ ]:
previous_answer_questions_stats.shape

## Merge data 

In [ ]:
data.head(10)

In [ ]:
%%time
data['key'] = data['key'].astype(str) + '-' + data['lu_previous_timestamp'].astype(str)

In [ ]:
%%time
#Keep only key and remove lu_previous_timestamp
data = data['key']
data = data.to_dict()

In [ ]:
%%time
keys, \
previous_false_answer, \
previous_correct_answer, \
nb_previous_questions_answered, \
previous_answer = previous_answer_questions_stats.values.T
#.sort_values(['key'])
del previous_answer_questions_stats
gc.collect()

In [ ]:
%%time
pfa_dict = dict(zip(keys, previous_false_answer))
pca_dict = dict(zip(keys, previous_correct_answer))
npqa_dict = dict(zip(keys, nb_previous_questions_answered))
pa_dict = dict(zip(keys, previous_answer))

In [ ]:
w = np.ndarray(shape=(len(data),), dtype=object)
x = np.ndarray(shape=(len(data),), dtype=object)
y = np.ndarray(shape=(len(data),), dtype=object)
z = np.ndarray(shape=(len(data),), dtype=object)

In [ ]:
%%time
for idx, v in (tqdm(data.items(), position=0, leave=True)):
        w[idx] = pfa_dict.get(v)
        x[idx] = pca_dict.get(v)
        y[idx] = npqa_dict.get(v)
        z[idx] = pa_dict.get(v)

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/previous_answer_questions_stats_final.npz',
                    previous_false_answer = w,
                    previous_correct_answer = x,
                    nb_previous_questions_answered = y,
                    previous_answer = z
                   )


In [ ]:
del w
del x
del y
del z
gc.collect()

In [ ]:
data.columns

In [ ]:
#Check Size of local variables
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

# Statistical features

# User stats

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['user_id', 'answered_correctly','key', 'timestamp']
data = load_data_np(file_path, columns,  "full_dataset")

In [ ]:
%%time
user_stats = data.groupby('user_id').agg({'answered_correctly': ['sum', 'count', 'skew', 'mean', 'std'] }).reset_index()

In [ ]:
%%time
user_stats.columns=user_stats.columns.map('_'.join)
user_stats.rename(columns = {
 'user_id_':'user_id'}, inplace = True)

In [ ]:
%%time
#Order 
user_stats = user_stats.sort_values(by=['user_id'])

In [ ]:
user_stats.shape

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
user_id = np.array(user_stats['user_id'].tolist())
answered_correctly_sum = np.array(data['answered_correctly_sum'].tolist())
answered_correctly_skew = np.array(user_stats['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(user_stats['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(user_stats['answered_correctly_std'].tolist())
answered_correctly_count = np.array(user_stats['answered_correctly_count'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/users_stats_final_to_predict.npz',
                    user_id = user_id,
                    user_answered_correctly_sum = answered_correctly_sum,
                    user_answered_correctly_skew = answered_correctly_skew,
                    user_answered_correctly_mean = answered_correctly_mean,
                    user_answered_correctly_std = answered_correctly_std,
                    user_

In [ ]:
user_stats.set_index('user_id', inplace=True, drop=True)

In [ ]:
%%time
data = merge__by_chunks_pandas(data = data,
                               data2 = user_stats,
                               key = 'user_id',
                               chunksize = 500000)
del user_stats
gc.collect()

In [ ]:
%%time
#Reorder data before extracting np arays
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
data.head(10)

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
answered_correctly_sum = np.array(data['answered_correctly_sum'].tolist())
answered_correctly_skew = np.array(data['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(data['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(data['answered_correctly_std'].tolist())
answered_correctly_count = np.array(data['answered_correctly_count'].tolist())
del data
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/users_stats_final.npz',
                    user_answered_correctly_sum = answered_correctly_sum,
                    user_answered_correctly_skew = answered_correctly_skew,
                    user_answered_correctly_mean = answered_correctly_mean,
                    user_answered_correctly_std = answered_correctly_std,
                    user_answered_correctly_count = answered_correctly_count
                   )
del answered_correctly_skew
del answered_correctly_mean
del answered_correctly_std
del answered_correctly_count
gc.collect()

# Content

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['content_id', 'answered_correctly', 'key', 'timestamp' ]
data = load_data_np(file_path, columns,  "full_dataset")

In [ ]:
%%time
content_id_stats = data.groupby('content_id').agg({'answered_correctly': ['skew', 'mean', 'std', 'median'] }).reset_index()

In [ ]:
%%time
content_id_stats.columns=content_id_stats.columns.map('_'.join)
content_id_stats.rename(columns = {
 'content_id_':'content_id'}, inplace = True)

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
content_id = np.array(content_id_stats['content_id'].tolist())
answered_correctly_skew = np.array(content_id_stats['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(content_id_stats['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(content_id_stats['answered_correctly_std'].tolist())
answered_correctly_median = np.array(content_id_stats['answered_correctly_median'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/content_stats_final_to_predict.npz',
                    content_id = content_id,
                    content_answered_correctly_skew = answered_correctly_skew,
                    content_answered_correctly_mean = answered_correctly_mean,
                    content_answered_correctly_std = answered_correctly_std,
                    content_answered_correctly_median = answered_correctly_median
                   )

In [ ]:
content_id_stats.set_index('content_id', inplace=True, drop=True)

In [ ]:
%%time
data = merge__by_chunks_pandas(data = data,
                               data2 = content_id_stats,
                               key = 'content_id',
                               chunksize = 500000)
del content_id_stats
gc.collect()

In [ ]:
%%time
#Reorder data before extracting np arays
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array  
answered_correctly_skew = np.array(data['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(data['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(data['answered_correctly_std'].tolist())
answered_correctly_median = np.array(data['answered_correctly_median'].tolist())
del data
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/content_stats_final.npz',
                    content_answered_correctly_skew = answered_correctly_skew,
                    content_answered_correctly_mean = answered_correctly_mean,
                    content_answered_correctly_std = answered_correctly_std,
                    content_answered_correctly_median = answered_correctly_median
                   )
del answered_correctly_skew
del answered_correctly_mean
del answered_correctly_std
del answered_correctly_median
gc.collect()

## User  - Answer

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['user_id','user_answer', 'correct_answer', 'answered_correctly','key', 'timestamp']
data = load_data_np(file_path, columns, 'full_dataset')

In [ ]:
%%time
user_stats = data.groupby(['user_id', 'user_answer']).agg({'answered_correctly': ['count','skew', 'mean', 'std', 'median'] }).reset_index()

In [ ]:
user_stats.head(20)

In [ ]:
%%time
user_stats.columns=user_stats.columns.map('_'.join)
user_stats.rename(columns = {
 'user_id_':'user_id', 
 'user_answer_':'user_answer'}, inplace = True)

In [ ]:
user_stats['answered_correctly_mean'].unique()

In [ ]:
%%time
data['key_anwser'] = data['user_id'].astype(str) + '-' + data['correct_answer'].astype(int).astype(str)

In [ ]:
%%time
user_stats['key_anwser'] = user_stats['user_id'].astype(str) + '-' + user_stats['user_answer'].astype(str)

In [ ]:
data['key_anwser'].head(10)

In [ ]:
user_stats['key_anwser'].head(10)

In [ ]:
user_stats.set_index('key_anwser', inplace=True, drop=True)

In [ ]:
%%time
data = merge__by_chunks_pandas(data = data,
                               data2 = user_stats,
                               key = 'key_anwser',
                               chunksize = 500000)
del user_stats
gc.collect()

In [ ]:
data.head(10)

In [ ]:
%%time
#Reorder data before extracting np arays
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
data.head(10)

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
answered_correctly_count = np.array(data['answered_correctly_count'].tolist())
answered_correctly_skew = np.array(data['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(data['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(data['answered_correctly_std'].tolist())
answered_correctly_median = np.array(data['answered_correctly_median'].tolist())
del data
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/answer_stats_final.npz',
                    user_answered_correctly_count = answered_correctly_count,
                    user_answered_correctly_skew = answered_correctly_skew,
                    user_answered_correctly_mean = answered_correctly_mean,
                    user_answered_correctly_std = answered_correctly_std,
                    user_answered_correctly_median = answered_correctly_median
                   )
del answered_correctly_sum
del answered_correctly_count
del answered_correctly_skew
del answered_correctly_mean
del answered_correctly_std
del answered_correctly_median
gc.collect()

## Content answer

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['content_id','user_answer', 'correct_answer', 'answered_correctly', 'key', 'timestamp' ]
data = load_data_np(file_path, columns)

In [ ]:
%%time
content_id_stats = data.groupby(['content_id', 'user_answer']).agg({'answered_correctly': ['count','skew', 'mean', 'std', 'median'] }).reset_index()

In [ ]:
content_id_stats.head(10)

In [ ]:
%%time
content_id_stats.columns=content_id_stats.columns.map('_'.join)
content_id_stats.rename(columns = {
 'content_id_':'content_id',
 'user_answer_':'user_answer'}, inplace = True)

In [ ]:
content_id_stats.head(10)

In [ ]:
%%time
data['key_anwser'] = data['content_id'].astype(str) + '-' + data['correct_answer'].astype(int).astype(str)

In [ ]:
%%time
content_id_stats['key_anwser'] = content_id_stats['content_id'].astype(str) + '-' + content_id_stats['user_answer'].astype(str)

In [ ]:
content_id_stats.set_index('key_anwser', inplace=True, drop=True)

In [ ]:
%%time
data = merge__by_chunks_pandas(data = data,
                               data2 = content_id_stats,
                               key = 'key_anwser',
                               chunksize = 500000)
del content_id_stats
gc.collect()

In [ ]:
%%time
#Reorder data before extracting np arays
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array 
answered_correctly_count = np.array(data['answered_correctly_count'].tolist())
answered_correctly_skew = np.array(data['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(data['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(data['answered_correctly_std'].tolist())
answered_correctly_median = np.array(data['answered_correctly_median'].tolist())
del data
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/answer_content_stats_final.npz',
                    content_answered_correctly_count = answered_correctly_count,
                    content_answered_correctly_skew = answered_correctly_skew,
                    content_answered_correctly_mean = answered_correctly_mean,
                    content_answered_correctly_std = answered_correctly_std,
                    content_answered_correctly_median = answered_correctly_median
                   )
del answered_correctly_count
del answered_correctly_skew
del answered_correctly_mean
del answered_correctly_std
del answered_correctly_median
gc.collect()

# Bundle

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['bundle_id', 'answered_correctly', 'key', 'timestamp' ]
data = load_data_np(file_path, columns, "full_dataset")

In [ ]:
%%time
bundle_id_stats = data.groupby('bundle_id').agg({'answered_correctly': ['skew', 'mean', 'std', 'median'] }).reset_index()

In [ ]:
%%time
bundle_id_stats.columns = bundle_id_stats.columns.map('_'.join)
bundle_id_stats.rename(columns = {
 'bundle_id_':'bundle_id'}, inplace = True)

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
bundle_id = np.array(bundle_id_stats['bundle_id'].tolist())
answered_correctly_skew = np.array(bundle_id_stats['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(bundle_id_stats['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(bundle_id_stats['answered_correctly_std'].tolist())
answered_correctly_median = np.array(bundle_id_stats['answered_correctly_median'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/bundle_stats_final_to_predict.npz',
                    bundle_id = bundle_id,
                    bundle_answered_correctly_skew = answered_correctly_skew,
                    bundle_answered_correctly_mean = answered_correctly_mean,
                    bundle_answered_correctly_std = answered_correctly_std,
                    bundle_answered_correctly_median = answered_correctly_median
                   )

In [ ]:
def merge__by_chunks_pandas(data, data2, key, chunksize):
    start = time.time()
    frames = {}
    chunk_size =  chunksize
    size_final_file = data.shape[0]
    cols = data.shape[1]
    nb_iterations = math.ceil((data.shape[0]/chunk_size))
    subset_start = 0
    print('Start iteractions over {} chunks to convert {:,} rows of data with {} columns'.format(nb_iterations, size_final_file, cols))
    print('Each chunk will process {:,} / {:,}'.format(chunksize, data.shape[0]))
    for i in tqdm(range(0,nb_iterations), position=0, leave=True):
        subset_end = subset_start + chunk_size
        print('subset_start : {}'.format(subset_start))
        print('subset_end : {}'.format(subset_end))
        print('Total Rows : {}'.format(data.shape[0]))
        if data.shape[0]<subset_end:
            subset_end = data.shape[0]
            print('subset end updated to remaining rows')
        if subset_end <= (data.shape[0]) :
            #data.set_index(key, inplace=True, drop=False)
            #data2.set_index(key, inplace=True, drop=False)
            subset = data[subset_start:subset_end]
            print('chunk #{} size : {:,} rows'.format(str(i + 1), subset.shape[0]))
            subset = subset.merge(data2, how = 'left', on=key)
            subset['chunk'] = i
            frames[i] = subset
            print ('data load for chunk {}'.format(i + 1))
            subset_start =  subset_end
            step = time.time()
            print( "Time to process data load for chunk {} :  \n {:.2f} min".format( i + 1, (step - start) / 60))
    print( "Starting Aggregate pandas dataframes")
    if len(frames) > 1:
        # Concat all new dataframes
        final = pd.concat(frames.values(), ignore_index=True)
    else:
        final = frames.values()
    print ("Finished")
    return final

In [ ]:
%%time
data = merge__by_chunks_pandas(data = data,
                               data2 = bundle_id_stats,
                               key = 'bundle_id',
                               chunksize = 500000)
del bundle_id_stats
gc.collect()

In [ ]:
%%time
#Reorder data before extracting np arays
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array  
answered_correctly_skew = np.array(data['answered_correctly_skew'].tolist())
answered_correctly_mean = np.array(data['answered_correctly_mean'].tolist())
answered_correctly_std = np.array(data['answered_correctly_std'].tolist())
answered_correctly_median = np.array(data['answered_correctly_median'].tolist())
del data
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/bundle_stats_final.npz',
                    bundle_answered_correctly_skew = answered_correctly_skew,
                    bundle_answered_correctly_mean = answered_correctly_mean,
                    bundle_answered_correctly_std = answered_correctly_std,
                    bundle_answered_correctly_median = answered_correctly_median
                   )
del answered_correctly_skew
del answered_correctly_mean
del answered_correctly_std
del answered_correctly_median
gc.collect()

# Lagtime

In [9]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = ['user_id', 'answered_correctly', 'key', 'timestamp' ]
lagtime_stats = load_data_np(file_path, columns, "full_dataset")

100%|██████████| 4/4 [00:50<00:00, 12.68s/it]

CPU times: user 27.7 s, sys: 18.4 s, total: 46.1 s
Wall time: 51.1 s


In [10]:
lagtime_stats = lagtime_stats.sort_values(by=['user_id', 'timestamp'])

In [11]:
lagtime_stats['p_timestamp'] = lagtime_stats.groupby('user_id')['timestamp'].shift()

In [12]:
lagtime_stats['previous_answer'] = lagtime_stats.groupby(['user_id'])['answered_correctly'].shift()

In [13]:
lagtime_stats.head(100)

,user_id,answered_correctly,key,timestamp,p_timestamp,previous_answer
7544707,115,1,115-5692,0,NaN,NaN
7544709,115,1,115-5716,56943,0.0,1.0
7544686,115,1,115-128,118363,56943.0,1.0
7544712,115,1,115-7860,131167,118363.0,1.0
7544721,115,1,115-7922,137965,131167.0,1.0
...,...,...,...,...,...,...
75159687,5382,1,5382-5000,0,NaN,NaN
75159658,5382,0,5382-3944,39828,0.0,1.0
75159646,5382,1,5382-217,132189,39828.0,0.0
75159708,5382,0,5382-5844,153727,132189.0,1.0


In [14]:
cum = lagtime_stats.groupby('user_id')['previous_answer'].agg(['cumsum', 'cumcount'])

In [15]:
lagtime_stats['user_answered_correctly_mean_real'] = cum['cumsum'] / cum['cumcount']
lagtime_stats['user_answered_correctly_count'] = cum['cumcount']
lagtime_stats['answered_correctly_user'] = cum['cumsum']
lagtime_stats.drop(columns=['previous_answer'], inplace=True)

In [16]:
lagtime_stats['user_answered_correctly_mean_real'].fillna(0, inplace=True)

In [17]:
lagtime_stats.head(20)

,user_id,answered_correctly,key,timestamp,p_timestamp,user_answered_correctly_mean_real,user_answered_correctly_count,answered_correctly_user
7544707,115,1,115-5692,0,NaN,0.000000,0,NaN
7544709,115,1,115-5716,56943,0.0,1.000000,1,1.0
7544686,115,1,115-128,118363,56943.0,1.000000,2,2.0
7544712,115,1,115-7860,131167,118363.0,1.000000,3,3.0
7544721,115,1,115-7922,137965,131167.0,1.000000,4,4.0
7544689,115,1,115-156,157063,137965.0,1.000000,5,5.0
7544705,115,1,115-51,176092,157063.0,1.000000,6,6.0
7544704,115,1,115-50,194190,176092.0,1.000000,7,7.0
7544716,115,1,115-7896,212463,194190.0,1.000000,8,8.0
7544713,115,1,115-7863,230983,212463.0,1.000000,9,9.0


In [18]:
lagtime_stats.head(20)

,user_id,answered_correctly,key,timestamp,p_timestamp,user_answered_correctly_mean_real,user_answered_correctly_count,answered_correctly_user
7544707,115,1,115-5692,0,NaN,0.000000,0,NaN
7544709,115,1,115-5716,56943,0.0,1.000000,1,1.0
7544686,115,1,115-128,118363,56943.0,1.000000,2,2.0
7544712,115,1,115-7860,131167,118363.0,1.000000,3,3.0
7544721,115,1,115-7922,137965,131167.0,1.000000,4,4.0
7544689,115,1,115-156,157063,137965.0,1.000000,5,5.0
7544705,115,1,115-51,176092,157063.0,1.000000,6,6.0
7544704,115,1,115-50,194190,176092.0,1.000000,7,7.0
7544716,115,1,115-7896,212463,194190.0,1.000000,8,8.0
7544713,115,1,115-7863,230983,212463.0,1.000000,9,9.0


In [19]:
#Check one user
lagtime_stats[lagtime_stats['user_id'] == 1509564249]\
          .sort_values(by=['timestamp'])

,user_id,answered_correctly,key,timestamp,p_timestamp,user_answered_correctly_mean_real,user_answered_correctly_count,answered_correctly_user
25609125,1509564249,0,1509564249-4463,0,NaN,0.000000,0,NaN
25615849,1509564249,0,1509564249-930,19079,0.000000e+00,0.000000,1,0.0
25612238,1509564249,1,1509564249-5880,45960,1.907900e+04,0.000000,2,0.0
25609552,1509564249,0,1509564249-4624,59209,4.596000e+04,0.333333,3,1.0
25609651,1509564249,0,1509564249-4710,83069,5.920900e+04,0.250000,4,1.0
...,...,...,...,...,...,...,...,...
25615635,1509564249,0,1509564249-9142,2974732924,2.974722e+09,0.220850,10283,2271.0
25613181,1509564249,0,1509564249-6281,2974744301,2.974733e+09,0.220828,10284,2271.0
25610561,1509564249,0,1509564249-5183,2974755454,2.974744e+09,0.220807,10285,2271.0
25607479,1509564249,0,1509564249-3857,2974766793,2.974755e+09,0.220786,10286,2271.0


In [20]:
lagtime_stats['p_timestamp'].fillna(0, inplace=True)

In [21]:
lagtime_stats['answered_correctly_user'].fillna(0, inplace=True)

In [22]:
lagtime_stats['lagtime'] = lagtime_stats['timestamp'] - lagtime_stats['p_timestamp']

In [23]:
lagtime_stats.drop(['p_timestamp', 'answered_correctly'], axis=1, inplace=True)

In [24]:
lagtime_stats['lagtime'].fillna(0, inplace=True)

In [25]:
lagtime_stats.head(20)

,user_id,answered_correctly,key,timestamp,user_answered_correctly_mean_real,user_answered_correctly_count,answered_correctly_user,lagtime
7544707,115,1,115-5692,0,0.000000,0,0.0,0.0
7544709,115,1,115-5716,56943,1.000000,1,1.0,56943.0
7544686,115,1,115-128,118363,1.000000,2,2.0,61420.0
7544712,115,1,115-7860,131167,1.000000,3,3.0,12804.0
7544721,115,1,115-7922,137965,1.000000,4,4.0,6798.0
7544689,115,1,115-156,157063,1.000000,5,5.0,19098.0
7544705,115,1,115-51,176092,1.000000,6,6.0,19029.0
7544704,115,1,115-50,194190,1.000000,7,7.0,18098.0
7544716,115,1,115-7896,212463,1.000000,8,8.0,18273.0
7544713,115,1,115-7863,230983,1.000000,9,9.0,18520.0


In [26]:
lagtime_stats.dtypes

user_id                                int64
answered_correctly                     int64
key                                   object
timestamp                              int64
user_answered_correctly_mean_real    float64
user_answered_correctly_count          int64
answered_correctly_user              float64
lagtime                              float64
dtype: object

In [27]:
lagtime_stats.lagtime = lagtime_stats.lagtime.astype('int64')
#lagtime_stats.p_timestamp = lagtime_stats.p_timestamp.astype('int64')
lagtime_stats.answered_correctly_user = lagtime_stats.answered_correctly_user.astype('int64')

In [ ]:
%%time
#Reorder data before extracting np arays
lagtime_stats = lagtime_stats.sort_values(by=['key', 'timestamp'])

In [ ]:
pd.set_option('display.max_rows', None)
lagtime_stats[lagtime_stats['user_id'] == 1000535709].sort_values(by=['timestamp'])

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array  
lagtime = np.array(lagtime_stats['lagtime'].tolist())
user_answered_correctly_mean_real = np.array(lagtime_stats['user_answered_correctly_mean_real'].tolist())
#p_timestamp = np.array(lagtime_stats['p_timestamp'].tolist())
answered_correctly_user = np.array(lagtime_stats['answered_correctly_user'].tolist())
user_answered_correctly_count = np.array(lagtime_stats['user_answered_correctly_count'].tolist())
del lagtime_stats
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/lagtime_stats_final.npz',
                    lagtime = lagtime,
                    #p_timestamp = p_timestamp,
                    user_answered_correctly_mean_real = user_answered_correctly_mean_real,
                    answered_correctly_user = answered_correctly_user, 
                    user_answered_correctly_count = user_answered_correctly_count
                   )
del lagtime
gc.collect()

# Look at question for timestamp 0

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = [ 'user_id', 'content_id', 'question_answered', 'answered_correctly', 'timestamp' ]
data = load_data_np(file_path, columns, 'full_dataset')

In [ ]:
first = data[data['timestamp'] == 0]

In [ ]:
pd.set_option('display.max_rows', None)
first.groupby(['content_id']).agg({'question_answered': 'count', 'answered_correctly':'mean' }).reset_index().sort_values(by='question_answered', ascending=False)

## nb questions progress 

In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz'
columns = [ 'user_id','part', 'content_id', 'question_answered', 'answered_correctly', 'key', 'timestamp' ]
data = load_data_np(file_path, columns, 'full_dataset')

In [ ]:
%%time
data['question_answered_progress'] = data.sort_values(by=['user_id', 'part', 'timestamp'], ascending=True)\
                       .groupby(['user_id'])['question_answered']\
                       .cumsum()

In [ ]:
data.head(10)

In [ ]:
%%time
data.loc[data.part == 1, 'question_answered_progress_bin_part_1' ] = data['question_answered_progress']
data.loc[data.part == 2, 'question_answered_progress_bin_part_2' ] = data['question_answered_progress']
data.loc[data.part == 3, 'question_answered_progress_bin_part_3' ] = data['question_answered_progress']
data.loc[data.part == 4, 'question_answered_progress_bin_part_4' ] = data['question_answered_progress']
data.loc[data.part == 5, 'question_answered_progress_bin_part_5' ] = data['question_answered_progress']
data.loc[data.part == 6, 'question_answered_progress_bin_part_6' ] = data['question_answered_progress']
data.loc[data.part == 7, 'question_answered_progress_bin_part_7' ] = data['question_answered_progress']

In [ ]:
data.head(10)

In [ ]:
%%time
data['question_answered_progress_bin_part_1'] = pd.qcut(data['question_answered_progress_bin_part_1'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
data['question_answered_progress_bin_part_2'] = pd.qcut(data['question_answered_progress_bin_part_2'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
data['question_answered_progress_bin_part_3'] = pd.qcut(data['question_answered_progress_bin_part_3'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
data['question_answered_progress_bin_part_4'] = pd.qcut(data['question_answered_progress_bin_part_4'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
data['question_answered_progress_bin_part_5'] = pd.qcut(data['question_answered_progress_bin_part_5'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
data['question_answered_progress_bin_part_6'] = pd.qcut(data['question_answered_progress_bin_part_6'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
data['question_answered_progress_bin_part_7'] = pd.qcut(data['question_answered_progress_bin_part_7'], q=10, labels=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

In [ ]:
%%time
#24min 6s
data.loc[data.part == 1, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_1']
data.loc[data.part == 2, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_2']
data.loc[data.part == 3, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_3']
data.loc[data.part == 4, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_4']
data.loc[data.part == 5, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_5']
data.loc[data.part == 6, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_6']
data.loc[data.part == 7, 'question_answered_progress_bin' ] = data['question_answered_progress_bin_part_7']

In [ ]:
data.head(10)

In [ ]:
%%time
question_answered_progress_stats = data.groupby(['part', 'question_answered_progress_bin']).agg({'answered_correctly': ['count','skew', 'mean', 'std'] }).reset_index()


In [ ]:
%%time
question_answered_progress_stats.columns=question_answered_progress_stats.columns.map('_'.join)
question_answered_progress_stats.rename(columns = {
 'question_answered_progress_bin_':'question_answered_progress_bin',
 'question_answered_progress_':'question_answered_progress',
 'part_':'part'}, inplace = True)

In [ ]:
def merge__by_chunks_pandas(data, data2, key, chunksize):
    start = time.time()
    frames = {}
    chunk_size =  chunksize
    size_final_file = data.shape[0]
    cols = data.shape[1]
    nb_iterations = math.ceil((data.shape[0]/chunk_size))
    subset_start = 0
    print('Start iteractions over {} chunks to convert {:,} rows of data with {} columns'.format(nb_iterations, size_final_file, cols))
    print('Each chunk will process {:,} / {:,}'.format(chunksize, data.shape[0]))
    for i in tqdm(range(0,nb_iterations), position=0, leave=True):
        subset_end = subset_start + chunk_size
        print('subset_start : {}'.format(subset_start))
        print('subset_end : {}'.format(subset_end))
        print('Total Rows : {}'.format(data.shape[0]))
        if data.shape[0]<subset_end:
            subset_end = data.shape[0]
            print('subset end updated to remaining rows')
        if subset_end <= (data.shape[0]) :
            #data.set_index(key, inplace=True, drop=False)
            #data2.set_index(key, inplace=True, drop=False)
            subset = data[subset_start:subset_end]
            print('chunk #{} size : {:,} rows'.format(str(i + 1), subset.shape[0]))
            subset = subset.merge(data2, how = 'left', on=key)
            subset['chunk'] = i
            frames[i] = subset
            print ('data load for chunk {}'.format(i + 1))
            subset_start =  subset_end
            step = time.time()
            print( "Time to process data load for chunk {} :  \n {:.2f} min".format( i + 1, (step - start) / 60))
    print( "Starting Aggregate pandas dataframes")
    if len(frames) > 1:
        # Concat all new dataframes
        final = pd.concat(frames.values(), ignore_index=True)
    else:
        final = frames.values()
    print ("Finished")
    return final

In [ ]:
%%time
data = merge__by_chunks_pandas(data = data,
                               data2 = question_answered_progress_stats,
                               key = 'question_answered_progress_bin',
                               chunksize = 500000)
#del question_answered_progress_stats
#gc.collect()

In [ ]:
data.head(10)

In [ ]:
%%time
#Reorder data before extracting np arays
data = data.sort_values(by=['key', 'timestamp'])

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array  
question_answered_progress = np.array(data['question_answered_progress'].tolist())
answered_correctly_mean = np.array(data['answered_correctly_mean'].tolist())
del data
gc.collect()

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/question_answered_progress.npz',
                    question_answered_progress = question_answered_progress,
                    answered_correctly_mean = answered_correctly_mean
                   )
del question_answered_progress
del answered_correctly_mean
gc.collect()

# Final DATA

In [ ]:
%%time
data = np.load('/Users/nlisch/downloads/Kaggle_riid/process/data_clean.npz' , allow_pickle=True)
previous_answer_questions_stats_final = np.load('/Users/nlisch/downloads/Kaggle_riid/process/previous_answer_questions_stats_final.npz' , allow_pickle=True)
content_stats = np.load('/Users/nlisch/downloads/Kaggle_riid/process/content_stats_final.npz' , allow_pickle=True)
users_stats = np.load('/Users/nlisch/downloads/Kaggle_riid/process/users_stats_final.npz' , allow_pickle=True)
answer_prob_final = np.load('/Users/nlisch/downloads/Kaggle_riid/process/answer_prob_final.npz' , allow_pickle=True)
correct_answer_stats = np.load('/Users/nlisch/downloads/Kaggle_riid/process/correct_answer_stats_final.npz' , allow_pickle=True)
answer_stats_final = np.load('/Users/nlisch/downloads/Kaggle_riid/process/answer_stats_final.npz' , allow_pickle=True)
answer_content_stats_final = np.load('/Users/nlisch/downloads/Kaggle_riid/process/answer_content_stats_final.npz' , allow_pickle=True)
question_answered_progress = np.load('/Users/nlisch/downloads/Kaggle_riid/process/question_answered_progress.npz' , allow_pickle=True)
data_on_the_fly = np.load('/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/questions_final_dummies_part1.npz' , allow_pickle=True)
data_on_the_fly2 = np.load('/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/questions_final_dummies_part2.npz' , allow_pickle=True)
pca = np.load('/Users/nlisch/downloads/Kaggle_riid/process/prediction/pca_test.npz', allow_pickle=True)
bundle_stats = np.load('/Users/nlisch/downloads/Kaggle_riid/process/bundle_stats_final.npz' , allow_pickle=True)
lagtime_stats = np.load('/Users/nlisch/downloads/Kaggle_riid/process/lagtime_stats_final.npz' , allow_pickle=True)


In [ ]:
%%time
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/content_stats_final.npz'
columns = ['content_answered_correctly_mean', 'content_answered_correctly_count']
staging_1 = load_data_np(file_path, columns, 'full_dataset')
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/users_stats_final.npz'
columns = ['user_answered_correctly_mean', 'user_answered_correctly_count']
staging_2 = load_data_np(file_path, columns, 'full_dataset')
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/correct_answer_stats_final.npz'
columns = ['correct_answer_correctly_mean']
staging_3 = load_data_np(file_path, columns, 'full_dataset')
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/answer_stats_final.npz'
columns = ['user_answered_correctly_mean']
staging_4 = load_data_np(file_path, columns, 'full_dataset')
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/answer_content_stats_final.npz'
columns = ['content_answered_correctly_mean']
staging_5 = load_data_np(file_path, columns, 'full_dataset')
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/question_answered_progress.npz'
columns = ['answered_correctly_mean']
staging_6 = load_data_np(file_path, columns, 'full_dataset')
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/lagtime_stats_final.npz'
columns = ['user_answered_correctly_mean_real']
staging_7 = load_data_np(file_path, columns, 'full_dataset')

In [ ]:
%%time
#Add addtionnale features
# add harmonic mean
staging_1['hmean_user_content_accuracy'] = 2 * (
    (staging_2['user_answered_correctly_mean'] * staging_1['content_answered_correctly_mean']) /
    (staging_2['user_answered_correctly_mean'] + staging_1['content_answered_correctly_mean'])
)

staging_1['hmean_user_answered_correctly_mean_real'] = 2 * (
    (staging_7['user_answered_correctly_mean_real'] * staging_5['content_answered_correctly_mean']) /
    (staging_7['user_answered_correctly_mean_real'] + staging_5['content_answered_correctly_mean'])
)


staging_1['hmean_answer_content_accuracy'] = 2 * (
    (staging_4['user_answered_correctly_mean'] * staging_5['content_answered_correctly_mean']) /
    (staging_4['user_answered_correctly_mean'] + staging_5['content_answered_correctly_mean'])
)


staging_1['hmean_user_content_accuracy_correct_answer'] = 3 * (
    (staging_2['user_answered_correctly_mean'] * staging_1['content_answered_correctly_mean'] * staging_3['correct_answer_correctly_mean'])/
    ((staging_2['user_answered_correctly_mean']*staging_1['content_answered_correctly_mean']) + (staging_1['content_answered_correctly_mean']*staging_3['correct_answer_correctly_mean']) + (staging_3['correct_answer_correctly_mean']*staging_2['user_answered_correctly_mean']))
)

staging_1['hmean_user_content_accuracy_progress_question'] = 3 * (
    (staging_2['user_answered_correctly_mean'] * staging_1['content_answered_correctly_mean'] * staging_6['answered_correctly_mean'])/
    ((staging_2['user_answered_correctly_mean']*staging_1['content_answered_correctly_mean']) + (staging_1['content_answered_correctly_mean']*staging_6['answered_correctly_mean']) + (staging_6['answered_correctly_mean']*staging_2['user_answered_correctly_mean']))
)

## Save final 

In [ ]:
%%time
#17min 38s with ky feature
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/data_features_final.npz',
                    
                    # Part 1
                    question_6116 = data_on_the_fly2['question_6116'],
                    question_6173 = data_on_the_fly2['question_6173'],
                    question_4120 = data_on_the_fly2['question_4120'],
                    question_175 = data_on_the_fly2['question_175'],
                    question_7876 = data_on_the_fly2['question_7876'],
                    question_7900 = data_on_the_fly2['question_7900'],
                    question_2065 = data_on_the_fly2['question_2065'],
                    question_2064 = data_on_the_fly2['question_2064'],
                    question_2063 = data_on_the_fly2['question_2063'],
                    question_4492 = data_on_the_fly2['question_4492'],
                    question_3365 = data_on_the_fly2['question_3365'],
                    question_3364 = data_on_the_fly2['question_3364'],
                    question_3363 = data_on_the_fly2['question_3363'],
                    question_4696 = data_on_the_fly2['question_4696'],
                    question_1278 = data_on_the_fly2['question_1278'],
                    question_6370 = data_on_the_fly2['question_6370'],
                    question_2946 = data_on_the_fly2['question_2946'],
                    question_2947 = data_on_the_fly2['question_2947'],
                    
                    
                    #From data file
                    #row_id = data['row_id'],
                    #key = data['key'],
                    timestamp = data['timestamp'],
                    user_id = data['user_id'],
                    content_id = data['content_id'],
                    #content_type_id = data['content_type_id'],
                    task_container_id = data['task_container_id'],
                    #user_answer = data['user_answer'],
                    answered_correctly = data['answered_correctly'],
                    prior_question_elapsed_time = data['prior_question_elapsed_time'],
                    #prior_question_had_explanation = data['prior_question_had_explanation'],
                    #question_id = data['question_id'],
                    bundle_id = data['bundle_id'],
                    #correct_answer = data['correct_answer'],
                    part = data['part'],
                    #tags = data['tags'],
                    question_answered = data['question_answered'],
                    #false_answer = data['false_answer'], 
                    #key= data['key'],
                    #From previous answered data
                    previous_false_answer = previous_answer_questions_stats_final['previous_false_answer'],
                    #previous_correct_answer = previous_answer_questions_stats_final['previous_correct_answer'],
                    #nb_previous_questions_answered = previous_answer_questions_stats_final['nb_previous_questions_answered'],
                    #previous_answer = previous_answer_questions_stats_final['previous_answer'],
                    #content_answered_correctly_skew = content_stats['content_answered_correctly_skew'],
                    content_answered_correctly_mean = content_stats['content_answered_correctly_mean'],
                    bundle_answered_correctly_mean = bundle_stats['bundle_answered_correctly_mean'],
                    #content_answered_correctly_std = content_stats['content_answered_correctly_std'],
                    #content_answered_correctly_median = content_stats['content_answered_correctly_median'],
                    #user_answered_correctly_skew = users_stats['user_answered_correctly_skew'],
                    user_answered_correctly_mean = users_stats['user_answered_correctly_mean'],
                    #user_answered_correctly_std = users_stats['user_answered_correctly_std'],
                    user_answered_correctly_count = users_stats['user_answered_correctly_count'],
                    #user_answer_prob_0 = answer_prob_final['user_answer_prob_0'],
                    #user_answer_prob_1 = answer_prob_final['user_answer_prob_1'],
                    #user_answer_prob_2 = answer_prob_final['user_answer_prob_2'],
                    #user_answer_prob_3 = answer_prob_final['user_answer_prob_3'],
                    #prob_answer_correct = answer_prob_final['prob_answer_correct'],
                    #prob_answer_incorrect = answer_prob_final['prob_answer_incorrect'],
                    hmean_user_content_accuracy = staging_1['hmean_user_content_accuracy'],
                    content_answered_correctly_count = content_stats['content_answered_correctly_count'],
                    #hmean_answer_content_accuracy = staging_1['hmean_answer_content_accuracy'],
                    answer_answered_correctly_mean = answer_stats_final['user_answered_correctly_mean'],
                    question_answered_progress = question_answered_progress['question_answered_progress'],
                    question_answered_progress_mean = question_answered_progress['answered_correctly_mean'],
                    #hmean_user_content_accuracy_progress_question = staging_1['hmean_user_content_accuracy_progress_question'],
                    answered_correctly_user = users_stats['user_answered_correctly_sum'],
                    lagtime_stats = lagtime_stats['lagtime'],
                    user_answered_correctly_mean_real = lagtime_stats['user_answered_correctly_mean_real'],
                    hmean_user_answered_correctly_mean_real = staging_1['hmean_user_answered_correctly_mean_real']
                   )
                

del data
del previous_answer_questions_stats_final
del content_stats
del users_stats
del answer_prob_final
gc.collect()

In [ ]:
%%time
#17min 38s with ky feature
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/data_features_final2.npz',

                    
                  #Specific
                    question_5692 = data_on_the_fly2['question_5692'],
                    question_128 = data_on_the_fly2['question_128'],
                    
                  # Part 3
                    question_6878 = data_on_the_fly2['question_6878'],
                    question_3731 = data_on_the_fly2['question_3731'],
                    question_4995 = data_on_the_fly2['question_4995'],
                    question_4414 = data_on_the_fly2['question_4414'],
                    question_1315 = data_on_the_fly2['question_1315'],
                    question_6908 = data_on_the_fly2['question_6908'],
                    question_6909 = data_on_the_fly2['question_6909'],
                    question_6911 = data_on_the_fly2['question_6911'],
                    question_6910 = data_on_the_fly2['question_6910'],
                    question_877 = data_on_the_fly2['question_877'],
                    question_3717 = data_on_the_fly2['question_3717'],
                    question_4476 = data_on_the_fly2['question_4476'],
                    question_853 = data_on_the_fly2['question_853'],
                    question_4108 = data_on_the_fly2['question_4108'],
                    question_951 = data_on_the_fly2['question_951'],
                    question_5177 = data_on_the_fly2['question_5177'],
                    question_6318 = data_on_the_fly2['question_6318'],
                    question_199 = data_on_the_fly2['question_199'],
                    question_7898 = data_on_the_fly2['question_7898'],
                    question_738 = data_on_the_fly2['question_738'],
                    
                    # Part 2
                    question_2594 = data_on_the_fly2['question_2594'],
                    question_2595 = data_on_the_fly2['question_2595'],
                    question_2593 = data_on_the_fly2['question_2593'],
                    question_7218 = data_on_the_fly2['question_7218'],
                    question_7217 = data_on_the_fly2['question_7217'],
                    question_7216 = data_on_the_fly2['question_7216'],
                    question_7219 = data_on_the_fly2['question_7219'],
                    question_10686 = data_on_the_fly2['question_10686'],
                    question_294 = data_on_the_fly2['question_294'],
                    question_10685 = data_on_the_fly2['question_10685'],
                    question_10688 = data_on_the_fly2['question_10688'],
                    question_10684 = data_on_the_fly2['question_10684'],
                    question_10687 = data_on_the_fly2['question_10687'],
                    question_3878 = data_on_the_fly2['question_3878'],
                    question_6879 = data_on_the_fly2['question_6879'],
                    question_6877 = data_on_the_fly2['question_6877'],
                    question_6880 = data_on_the_fly2['question_6880']
                   )

In [ ]:
%%time
#17min 38s with ky feature
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/data_features_final3.npz',
                    

                    # Part 5
                    question_2948 = data_on_the_fly2['question_2948'],
                    
                    # Part 4
                    question_4821 = data_on_the_fly2['question_4821'],
                    question_824 = data_on_the_fly2['question_824'],
                    question_6117 = data_on_the_fly2['question_6117'],
                    question_5616 = data_on_the_fly2['question_5616'],
                    question_446 = data_on_the_fly2['question_446'],
                    question_6120 = data_on_the_fly2['question_6120'],
                    question_834 = data_on_the_fly2['question_834'],
                    question_9259 = data_on_the_fly2['question_9259'],
                    question_762 = data_on_the_fly2['question_762'],
                    question_6002 = data_on_the_fly2['question_6002'],
                    question_4258 = data_on_the_fly2['question_4258'],
                    question_5703 = data_on_the_fly2['question_5703'],
                    question_6119 = data_on_the_fly2['question_6119'],
                    question_638 = data_on_the_fly2['question_638'],
                    question_405 = data_on_the_fly2['question_405'],
                    question_1313 = data_on_the_fly2['question_1313'],
                    question_4416 = data_on_the_fly2['question_4416'],
                    question_6668 = data_on_the_fly2['question_6668'],
                    question_3871 = data_on_the_fly2['question_3871'],
                    question_4340 = data_on_the_fly2['question_4340'],
                    question_1296 = data_on_the_fly2['question_1296'],
                    question_4966 = data_on_the_fly2['question_4966'],
                   )

In [ ]:
x = np.load('/Users/nlisch/downloads/Kaggle_riid/process/data_features_final.npz', mmap_mode='r')
for k in x.files:
    print(k)

# CALCULATE Previous list correct anf false for model + list content ID

# previous stats

In [ ]:
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/previous_answer_questions_to_predict.npz'
columns = ['user_id', 'content_id',  'previous_false_answer', 'previous_correct_answer']

data = load_data_np(file_path, columns, 'previous_stats_questions' )

In [ ]:
%%time
data.fillna(0, inplace=True)

In [ ]:
%%time
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/content_ids_to_predict.npz'
columns = ['user_id',  'count_question_anwsered', 'answered_correctly_user']

data2 = load_data_np(file_path, columns, 'user_stats' )



In [ ]:
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/users_stats_final_to_predict.npz'
columns = ['user_id',
           'user_answered_correctly_mean']

data3 = load_data_np(file_path, columns, 'user_stats')

# New method

In [ ]:
data.head(1)

In [ ]:
users_list = data['user_id'].unique().tolist()

In [ ]:
%%time
dict_map = {}
j = 0
for i in tqdm(range(10000, 410000, 10000)):
    print(i)
    list_it = (users_list[j:i])
    for k in list_it:
        dict_map[k] = i
    file_name = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/user_state/part_' + str(i) + '.npz'
    subset = data[data['user_id'].isin(list_it)]
    subset = subset.merge(data2, how = 'left', on = 'user_id')
    subset = subset.merge(data3, how = 'left', on = 'user_id')
    print(subset.head(10))
    user_id = np.array(subset['user_id'].tolist())
    print(user_id)
    content_id = np.array(subset['content_id'].tolist())
    previous_false_answer = np.array(subset['previous_false_answer'].tolist())
    previous_correct_answer = np.array(subset['previous_correct_answer'].tolist())
    count_question_anwsered = np.array(subset['count_question_anwsered'].tolist())
    answered_correctly_user = np.array(subset['answered_correctly_user'].tolist())
    user_answered_correctly_mean = np.array(subset['user_answered_correctly_mean'].tolist())
    print(file_name)
    np.savez_compressed(file_name,
                    user_id = user_id,
                    content_id = content_id,
                    previous_false_answer = previous_false_answer,
                    previous_correct_answer = previous_correct_answer,
                    user_answered_correctly_count = count_question_anwsered, 
                    answered_correctly_user = answered_correctly_user,
                    user_answered_correctly_mean = user_answered_correctly_mean
                        
                   )
    j = i

In [ ]:
len(dict_map)

In [ ]:
import json
json = json.dumps(dict_map)
f = open("dict_map.json","w")
f.write(json)
f.close()

In [ ]:
%%time
import os
import zipfile

def zipdir(path, ziph):
    lenDirPath = len(path)
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        
        for file in files:
            filePath = os.path.join(root, file)
            ziph.write(filePath, filePath[lenDirPath :])

if __name__ == '__main__':
    zipf = zipfile.ZipFile('/Users/nlisch/downloads/Kaggle_riid/process/test_zip/state.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('/Users/nlisch/downloads/Kaggle_riid/process/prediction/user_state/', zipf)
    zipf.close()

In [ ]:
%%time
for i in tqdm(users_list[67002:]):
    file_name = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/users_state/' + str(i) + '.npz'
    print(i)
    subset = data[data['user_id'].isin([i])]
    user_id = np.array(subset['user_id'].tolist())
    content_id = np.array(subset['content_id'].tolist())
    previous_false_answer = np.array(subset['previous_false_answer'].tolist())
    previous_correct_answer = np.array(subset['previous_correct_answer'].tolist())
    np.savez_compressed(file_name,
                    user_id = user_id,
                    content_id = content_id,
                    previous_false_answer = previous_false_answer,
                    previous_correct_answer = previous_correct_answer,
                   )

# Check Load again

In [ ]:
N_ROWS = 99271101
N_ROWS_train = 50004223
N_ROWS_test = 2500267
N_ROWS_user_stats = 393656
N_ROWS_content_stats = 13523
N_ROWS_previous_stats_questions = 86867031
def get_index_np(type_data):
    if type_data == 'train':
        final = np.arange(N_ROWS_train)
    if type_data == 'test':
        final = np.arange(N_ROWS_test)
    if type_data == 'user_stats':
        final = np.arange(N_ROWS_user_stats)
    if type_data == 'content_stats':
        final = np.arange(N_ROWS_content_stats)
    if type_data == 'previous_stats_questions':
        final = np.arange(N_ROWS_previous_stats_questions)
    if type_data == 'full_dataset':
        final = np.arange(N_ROWS)
    print(final)
    return final

In [ ]:
def load_data_np(file_path, columns_selected):
    data = np.load(file_path , allow_pickle=True)
    df = pd.DataFrame(index=np.arange(data['user_id'].shape[0]
))
    for col in tqdm(columns_selected):
        df[col] = data[col]
    return df

In [ ]:
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/users_state/2147482888.npz'
columns = ['user_id', 'content_id',  'previous_false_answer', 'previous_correct_answer']

user = load_data_np(file_path, columns )

# Old method

In [ ]:
%%time
#Order to predict
data = data.sort_values(by=['user_id'])

In [ ]:
pf_list = data.groupby(['user_id', 'content_id'])['previous_false_answer'].sum().astype(np.uint8).groupby('user_id').apply(np.array).values

In [ ]:
pc_list = data.groupby(['user_id', 'content_id'])['previous_correct_answer'].sum().astype(np.uint8).groupby('user_id').apply(np.array).values

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/previous_arrays_to_predict.npz',
                    pf_list = pf_list,
                    pc_list = pc_list
                   )

## unique content by user

In [ ]:
#CREATE FIRST QUESTION FEATURES

In [ ]:
%%time
N_ROWS = 99271101

def get_index_np():
    return np.arange(N_ROWS)
def load_data_np_multifiles(file_path, file_path2, file_path3, columns_selected, columns_selected2, columns_selected3):
    data = np.load(file_path , allow_pickle=True)
    data2 = np.load(file_path2 , allow_pickle=True)
    data3 = np.load(file_path3 , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np())
    for col in tqdm(columns_selected):
        df[col] = data[col]
    for col in tqdm(columns_selected2):
        df[col] = data2[col]
    for col in tqdm(columns_selected3):
        df[col] = data3[col]
    return df


#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final.npz'
file_path2 = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final2.npz'
file_path3 = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final3.npz'
features = ['timestamp','user_id']

features2 = [
'question_6116',
'question_6173',
'question_4120',
'question_175',
'question_7876',
'question_7900',
'question_2065',
'question_2064',
'question_2063',
'question_4492',
'question_3365',
'question_3364',
'question_3363',
'question_4696',
'question_1278',
'question_6370',
'question_2946',
'question_2947',
'question_6878',
'question_3731',
'question_4995',
'question_4414',
'question_1315',
'question_6908',
'question_6909',
'question_6911',
'question_6910',
'question_877',

            ]

features3 = []


data = load_data_np_multifiles(file_path,
                    file_path2,
                    file_path3,
                    features,
                   features2, 
                   features3
                              )
#Build features list
features = features + features2 + features3
print(len(features))

In [ ]:
%%time
data = data.sort_values(by=['user_id', 'timestamp'], ascending=False)

In [ ]:
%%time
data.drop_duplicates(subset=['user_id'], inplace = True)

In [ ]:
question_6116 = np.array(data['question_6116'].tolist())
question_6173 = np.array(data['question_6173'].tolist())
question_4120 = np.array(data['question_4120'].tolist())
question_175 = np.array(data['question_175'].tolist())
question_7876 = np.array(data['question_7876'].tolist())
question_7900 = np.array(data['question_7900'].tolist())
question_2065 = np.array(data['question_2065'].tolist())
question_2064 = np.array(data['question_2064'].tolist())
question_2063 = np.array(data['question_2063'].tolist())
question_4492 = np.array(data['question_4492'].tolist())
question_3365 = np.array(data['question_3365'].tolist())
question_3364 = np.array(data['question_3364'].tolist())
question_3363 = np.array(data['question_3363'].tolist())
question_4696 = np.array(data['question_4696'].tolist())
question_1278 = np.array(data['question_1278'].tolist())
question_6370 = np.array(data['question_6370'].tolist())
question_2946 = np.array(data['question_2946'].tolist())
question_6878 = np.array(data['question_6878'].tolist())
question_3731 = np.array(data['question_3731'].tolist())
question_4995 = np.array(data['question_4995'].tolist())
question_4414 = np.array(data['question_4414'].tolist())
question_1315 = np.array(data['question_1315'].tolist())
question_6908 = np.array(data['question_6908'].tolist())
question_6909 = np.array(data['question_6909'].tolist())
question_6911 = np.array(data['question_6911'].tolist())
question_6910 = np.array(data['question_6910'].tolist())
question_877 = np.array(data['question_877'].tolist())


In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/question_features_to_predict1.npz',
            question_6116 = question_6116,
question_6173 = question_6173,
question_4120 = question_4120,
question_175 = question_175,
question_7876 = question_7876,
question_7900 = question_7900,
question_2065 = question_2065,
question_2064 = question_2064,
question_2063 = question_2063,
question_4492 = question_4492,
question_3365 = question_3365,
question_3364 = question_3364,
question_3363 = question_3363,
question_4696 = question_4696,
question_1278 = question_1278,
question_6370 = question_6370,
question_2946 = question_2946,
question_6878 = question_6878,
question_3731 = question_3731,
question_4995 = question_4995,
question_4414 = question_4414,
question_1315 = question_1315,
question_6908 = question_6908,
question_6909 = question_6909,
question_6911 = question_6911,
question_6910 = question_6910,
question_877 = question_877,
                   )

In [ ]:
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final.npz'
file_path2 = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final2.npz'
file_path3 = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final3.npz'
features = ['timestamp','user_id']

features2 = [

'question_3717',
'question_4476',
'question_853',
'question_4108',
'question_951',
'question_5177',
'question_6318',
'question_199',
'question_7898',
'question_738',
'question_2594',
'question_2595',
'question_2593',
'question_7218',
'question_7217',
'question_7216',
'question_7219',
'question_10686',
'question_294',
'question_10685',
'question_10688',
'question_10684',
'question_10687',
'question_3878',
'question_6879',
'question_6877',
'question_6880'
            ]

features3 = []


data = load_data_np_multifiles(file_path,
                    file_path2,
                    file_path3,
                    features,
                   features2, 
                   features3
                              )
#Build features list
features = features + features2 + features3
print(len(features))

In [ ]:

question_3717 = np.array(data['question_3717'].tolist())
question_4476 = np.array(data['question_4476'].tolist())
question_853 = np.array(data['question_853'].tolist())
question_4108 = np.array(data['question_4108'].tolist())
question_951 = np.array(data['question_951'].tolist())
question_5177 = np.array(data['question_5177'].tolist())
question_6318 = np.array(data['question_6318'].tolist())
question_199 = np.array(data['question_199'].tolist())
question_7898 = np.array(data['question_7898'].tolist())
question_738 = np.array(data['question_738'].tolist())
question_2594 = np.array(data['question_2594'].tolist())
question_2595 = np.array(data['question_2595'].tolist())
question_2593 = np.array(data['question_2593'].tolist())
question_7218 = np.array(data['question_7218'].tolist())
question_7217 = np.array(data['question_7217'].tolist())
question_7216 = np.array(data['question_7216'].tolist())
question_7219 = np.array(data['question_7219'].tolist())
question_10686 = np.array(data['question_10686'].tolist())
question_294 = np.array(data['question_294'].tolist())
question_10685 = np.array(data['question_10685'].tolist())
question_10688 = np.array(data['question_10688'].tolist())
question_10684 = np.array(data['question_10684'].tolist())
question_10687 = np.array(data['question_10687'].tolist())
question_3878 = np.array(data['question_3878'].tolist())
question_6879 = np.array(data['question_6879'].tolist())
question_6877 = np.array(data['question_6877'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/question_features_to_predict2.npz',
            question_3717 = question_3717,
            question_4476 = question_4476,
            question_853 = question_853,
            question_4108 = question_4108,
            question_951 = question_951,
            question_5177 = question_5177,
            question_6318 = question_6318,
            question_199 = question_199,
            question_7898 = question_7898,
            question_738 = question_738,
            question_2594 = question_2594,
            question_2595 = question_2595,
            question_2593 = question_2593,
            question_7218 = question_7218,
            question_7217 = question_7217,
            question_7216 = question_7216,
            question_7219 = question_7219,
            question_10686 = question_10686,
            question_294 = question_294,
            question_10685 = question_10685,
            question_10688 = question_10688,
            question_10684 = question_10684,
            question_10687 = question_10687,
            question_3878 = question_3878,
            question_6879 = question_6879,
            question_6877 = question_6877,
            question_6880 = question_6880,
                   )

# Create content

In [ ]:
%%time
N_ROWS = 99271101
N_ROWS_train = 50004223
N_ROWS_test = 2500267
N_ROWS_user_stats = 393656
N_ROWS_content_stats = 13523
N_ROWS_previous_stats_questions = 86867031
def get_index_np(type_data):
    if type_data == 'train':
        final = np.arange(N_ROWS_train)
    if type_data == 'test':
        final = np.arange(N_ROWS_test)
    if type_data == 'user_stats':
        final = np.arange(N_ROWS_user_stats)
    if type_data == 'content_stats':
        final = np.arange(N_ROWS_content_stats)
    if type_data == 'previous_stats_questions':
        final = np.arange(N_ROWS_previous_stats_questions)
    if type_data == 'full_dataset':
        final = np.arange(N_ROWS)
    return final
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final.npz'
columns = ['user_id', 'content_id', 'user_answered_correctly_count', 'answered_correctly']

data = load_data_np(file_path, columns, 'full_dataset' )

In [ ]:
%%time
N_ROWS = 393656

def get_index_np():
    return np.arange(N_ROWS)
def load_data_np_multifiles(file_path, file_path2, file_path3, columns_selected, columns_selected2, columns_selected3):
    data = np.load(file_path , allow_pickle=True)
    data2 = np.load(file_path2 , allow_pickle=True)
    data3 = np.load(file_path3 , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np())
    for col in tqdm(columns_selected):
        df[col] = data[col]
    for col in tqdm(columns_selected2):
        df[col] = data2[col]
    for col in tqdm(columns_selected3):
        df[col] = data3[col]
    return df


#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/question_features_to_predict1.npz'
file_path2 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/question_features_to_predict2.npz'
file_path3 = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final3.npz'
features = [
'question_6116',
'question_6173',
'question_4120',
'question_175',
'question_7876',
'question_7900',
'question_2065',
'question_2064',
'question_2063',
'question_4492',
'question_3365',
'question_3364',
'question_3363',
'question_4696',
'question_1278',
'question_6370',
'question_2946',
'question_2947',
'user_id', 


           ]


features2 = [
'question_6878',
'question_3731',
'question_4995',
'question_4414',
'question_1315',
'question_6908',
'question_6909',
'question_6911',
'question_6910',
'question_877',
'question_3717',
'question_4476',
'question_853',
'question_4108',
'question_951',
'question_5177',
'question_6318',
'question_199',
'question_7898',
'question_738',
'question_2594',
'question_2595',
'question_2593',
'question_7218',
'question_7217',
'question_7216',
'question_7219',
'question_10686',
'question_294',
'question_10685',
'question_10688',
'question_10684',
'question_10687',
'question_3878',
'question_6879',
'question_6877',
'question_6880'
            ]

features3 = [
# 'question_2948',
#'question_4821',
#'question_824',
#'question_6117',
#'question_5616',
#'question_446',
#'question_6120',
#'question_834',
#'question_9259',
#'question_762',
#'question_6002',
#'question_4258',
#'question_5703',
#'question_6119',
#'question_638',
#'question_405',
#'question_1313',
#'question_4416',
#'question_6668',
#'question_3871',
#'question_4340',
#'question_1296',
#'question_4966',
]


data2 = load_data_np_multifiles(file_path,
                    file_path2,
                    file_path3,
                    features,
                   features2, 
                   features3
                              )
#Build features list
features = features + features2 + features3
print(len(features))

In [ ]:
%%time
#Order to predict
data2 = data2.sort_values(by=['user_id'])

In [ ]:
%%time
#Order to predict
data = data.sort_values(by=['user_id'])

In [ ]:
%%time
content_ids = data.groupby('user_id')['content_id'].apply(np.array).apply(np.sort).apply(np.unique)

In [ ]:
%%time
count_question_anwsered = data.groupby('user_id')['user_answered_correctly_count'].mean().values 

In [ ]:
%%time
answered_correctly_user = data.groupby('user_id')['answered_correctly'].sum().values

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/content_ids_to_predict.npz',
                    user_id = content_ids.index,
                    content_ids = content_ids, 
                    count_question_anwsered = count_question_anwsered,
                    answered_correctly_user = answered_correctly_user,
                    question_6116 = data2['question_6116'],
                    question_6173 = data2['question_6173'],
                    question_4120 = data2['question_4120'],
                    question_175 = data2['question_175'],
                    question_7876 = data2['question_7876'],
                    question_7900 = data2['question_7900'],
                    question_2065 = data2['question_2065'],
                    question_2064 = data2['question_2064'],
                    question_2063 = data2['question_2063'],
                    question_4492 = data2['question_4492'],
                    question_3365 = data2['question_3365'],
                    question_3364 = data2['question_3364'],
                    question_3363 = data2['question_3363'],
                    question_4696 = data2['question_4696'],
                    question_1278 = data2['question_1278'],
                    question_6370 = data2['question_6370'],
                    question_2946 = data2['question_2946'],
                    question_6878 = data2['question_6878'],
                    question_3731 = data2['question_3731'],
                    question_4995 = data2['question_4995'],
                    question_4414 = data2['question_4414'],
                    question_1315 = data2['question_1315'],
                    question_6908 = data2['question_6908'],
                    question_6909 = data2['question_6909'],
                    question_6911 = data2['question_6911'],
                    question_6910 = data2['question_6910'],
                    question_877 = data2['question_877'],
                    question_3717 = data2['question_3717'],
                    question_4476 = data2['question_4476'],
                    question_853 = data2['question_853'],
                    question_4108 = data2['question_4108'],
                    question_951 = data2['question_951'],
                    question_5177 = data2['question_5177'],
                    question_6318 = data2['question_6318'],
                    question_199 = data2['question_199'],
                    question_7898 = data2['question_7898'],
                    question_738 = data2['question_738'],
                    question_2594 = data2['question_2594'],
                    question_2595 = data2['question_2595'],
                    question_2593 = data2['question_2593'],
                    question_7218 = data2['question_7218'],
                    question_7217 = data2['question_7217'],
                    question_7216 = data2['question_7216'],
                    question_7219 = data2['question_7219'],
                    question_10686 = data2['question_10686'],
                    question_294 = data2['question_294'],
                    question_10685 = data2['question_10685'],
                    question_10688 = data2['question_10688'],
                    question_10684 = data2['question_10684'],
                    question_10687 = data2['question_10687'],
                    question_3878 = data2['question_3878'],
                    question_6879 = data2['question_6879'],
                    question_6877 = data2['question_6877'],
                    question_6880 = data2['question_6880'],
                    question_2947 = data2['question_2947'],
                    question_128 = data2['question_128'],
                    question_5692 = data2['question_5692'],
                   )